# Sprint9課題 アンサンブル学習

### 小さなデータセットの用意
以前も利用した回帰のデータセットを用意します。

[House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使います。

train.csvを学習用（train）8割、検証用（val）2割に分割してください。

### scikit-learn
単一のモデルはスクラッチ実装ではなく、scikit-learnなどのライブラリの使用を推奨します。

[sklearn.linear_model.LinearRegression — scikit-learn 0.20.0 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

[sklearn.svm.SVR — scikit-learn 0.20.0 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html)

[sklearn.tree.DecisionTreeRegressor — scikit-learn 0.20.0 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)

In [1]:
# モジュールをインポート
import numpy as np      #numpy
import pandas as pd      #pandas
import matplotlib.pyplot as plt      #グラフ
import seaborn as sns      #seaborn
%matplotlib inline
from sklearn.model_selection import train_test_split      #学習データ分割
from sklearn.linear_model import LinearRegression      #線形回帰
from sklearn.metrics import mean_squared_error      #平均二乗誤差
from sklearn.preprocessing import StandardScaler      #標準化ライブラリ
from sklearn.svm import SVR     #SVM(回帰)
from sklearn.tree import DecisionTreeRegressor   #決定木



In [2]:
# train.csvの読み込み
df = pd.read_csv("train.csv")

In [3]:
#GrLivAreaとYearBuiltを抜き出す
X = df.loc[:, ['GrLivArea', 'YearBuilt']].values

#目的変数SalePriceを抜き出す
y = df.loc[:, ['SalePrice']].values

In [4]:
#取り込みデータを学習用、検証用に分割する
X_train, X_validation, y_train, y_validation = train_test_split(
    X, y, test_size=0.2, random_state=42)

## 【問題1】ブレンディング
ブレンディングを実装し、単一モデルより精度があがる例を最低3つ示してください。精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを示します。

ブレンディングとは、N個の多様なモデルを独立して学習させ、推定結果を重み付けした上で足し合わせる方法です。最も単純には平均をとります。多様なモデルとは、以下のような条件を変化させることで作り出すものです。

- 手法（例：線形回帰、SVM、決定木、ニューラルネットワークなど）
- ハイパーパラメータ（例：SVMのカーネルの種類、重みの初期値など）
- 入力データの前処理の仕方（例：標準化、対数変換、PCAなど）


重要なのはそれぞれのモデルが大きく異なることです。必ずしも単一モデルの精度が高い必要はありません。

回帰問題でのブレンディングは非常に単純であるため、scikit-learnには用意されていません。

**補足**

分類問題の場合は、多数決を行います。回帰問題に比べると複雑なため、scikit-learnにはVotingClassifierが用意されています。

[sklearn.ensemble.VotingClassifier — scikit-learn 0.20.0 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html)

**考察**

どういった組み合わせが良いか、どのようにすると多様なモデルが作れるかを考えてみましょう。

### 1.データの前処理なし・パラメータデフォルトにてブレンディング

In [5]:
#線形回帰にて検証用データの予測
lr1 = LinearRegression()
lr1.fit(X_train, y_train)
y_val_pred_lr1 = lr1.predict(X_validation)

# 検証用データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_lr1))
))

[RMSE] 検証用データ : 49,955.529


In [6]:
#SVMにて検証用データの予測
svr1 = SVR(gamma='auto')
svr1.fit(X_train, y_train.reshape(-1))
y_val_pred_svr1 = svr1.predict(X_validation)
y_val_pred_svr1 = y_val_pred_svr1.reshape(y_val_pred_svr1.shape[0],1)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_svr1))
))

[RMSE] 検証用データ : 88,667.101


In [7]:
#決定木にて検証用データの予測
dtr1 = DecisionTreeRegressor()
dtr1.fit(X_train, y_train)
y_val_pred_dtr1 = dtr1.predict(X_validation)
y_val_pred_dtr1 = y_val_pred_dtr1.reshape(y_val_pred_dtr1.shape[0],1)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_dtr1))
))


[RMSE] 検証用データ : 47,516.068


In [8]:
#データの前処理なし・パラメータデフォルトにてブレンディング
y_val_pred_blend1 = np.mean([
    y_val_pred_lr1, 
    y_val_pred_svr1, 
    y_val_pred_dtr1
], axis=0)

In [9]:
#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_blend1))
))

[RMSE] 検証用データ : 52,131.358


**結果：精度は高まらず(線形回帰、決定木より精度が低い)**

### 2.データの前処理なし・SVMのみパラメータをチューニング

In [10]:
#線形回帰にて検証用データの予測
lr2 = LinearRegression()
lr2.fit(X_train, y_train)
y_val_pred_lr2 = lr2.predict(X_validation)

# 検証用データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_lr2))
))

[RMSE] 検証用データ : 49,955.529


In [11]:
#SVMにて検証用データの予測
svr2 = SVR(gamma='auto', kernel='linear', C=1e3, epsilon=46)
svr2.fit(X_train, y_train.reshape(-1))
y_val_pred_svr2 = svr2.predict(X_validation)
y_val_pred_svr2 = y_val_pred_svr2.reshape(y_val_pred_svr2.shape[0],1)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_svr2))
))

[RMSE] 検証用データ : 51,798.697


In [12]:
#決定木にて検証用データの予測
dtr2 = DecisionTreeRegressor()
dtr2.fit(X_train, y_train)
y_val_pred_dtr2 = dtr2.predict(X_validation)
y_val_pred_dtr2 = y_val_pred_dtr2.reshape(y_val_pred_dtr2.shape[0],1)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_dtr2))
))


[RMSE] 検証用データ : 48,598.143


In [13]:
#ブレンディング
y_val_pred_blend2 = np.mean([
    y_val_pred_lr2, 
    y_val_pred_svr2, 
    y_val_pred_dtr2
], axis=0)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_blend2))
))

[RMSE] 検証用データ : 45,721.635


**結果：精度が高まった**

### 3.データを標準化する(条件は上記2を利用)

In [14]:
#float型に変換
X_train = X_train.astype(np.float)
X_validation = X_validation.astype(np.float)
y_train = y_train.astype(np.float)
y_validation = y_validation.astype(np.float)


#学習データにて標準化ライブラリのインスタンス生成
scaler_X = StandardScaler()
scaler_X.fit(X_train)

#学習データ、検証データをそれぞれ標準化
X_train_st = scaler_X.transform(X_train)
X_validation_st = scaler_X.transform(X_validation)

#学習データにて標準化ライブラリのインスタンス生成
scaler_y = StandardScaler()
scaler_y.fit(y_train)

#学習データ、検証データをそれぞれ標準化
y_train_st = scaler_y.transform(y_train)
y_validation_st = scaler_y.transform(y_validation)

In [15]:
#線形回帰にて検証用データの予測
lr3 = LinearRegression()
lr3.fit(X_train_st, y_train)
y_val_pred_lr3 = lr3.predict(X_validation_st)

# 検証用データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_lr3))
))

[RMSE] 検証用データ : 49,955.529


In [16]:
#SVMにて検証用データの予測
svr3 = SVR(gamma='auto', kernel='linear', C=1e3, epsilon=46)
svr3.fit(X_train_st, y_train.reshape(-1))
y_val_pred_svr3 = svr3.predict(X_validation_st)
y_val_pred_svr3 = y_val_pred_svr3.reshape(y_val_pred_svr3.shape[0],1)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_svr3))
))

[RMSE] 検証用データ : 52,679.924


In [17]:
#決定木にて検証用データの予測
dtr3 = DecisionTreeRegressor()
dtr3.fit(X_train_st, y_train)
y_val_pred_dtr3 = dtr3.predict(X_validation_st)
y_val_pred_dtr3 = y_val_pred_dtr3.reshape(y_val_pred_dtr3.shape[0],1)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_dtr3))
))


[RMSE] 検証用データ : 47,929.345


In [18]:
#ブレンディング
y_val_pred_blend3 = np.mean([
    y_val_pred_lr3, 
    y_val_pred_svr3, 
    y_val_pred_dtr3
], axis=0)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_blend3))
))

[RMSE] 検証用データ : 45,734.026


**結果：精度が高まった**

### 4.データを対数変換する(条件は上記2を利用)

In [19]:
#特徴量データを対数変換する
X_train_log = np.log(X_train)
X_validation_log = np.log(X_validation)

#特徴量データを対数変換する
y_train_log = np.log(y_train)
y_validation_log = np.log(y_validation)

In [20]:
#線形回帰にて検証用データの予測
lr4 = LinearRegression()
lr4.fit(X_train_log, y_train)
y_val_pred_lr4 = lr4.predict(X_validation_log)

# 検証用データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_lr4))
))

[RMSE] 検証用データ : 54,530.346


In [21]:
#SVMにて検証用データの予測
svr4 = SVR(gamma='auto', kernel='linear', C=1e3, epsilon=46)
svr4.fit(X_train_log, y_train.reshape(-1))
y_val_pred_svr4 = svr4.predict(X_validation_log)
y_val_pred_svr4 = y_val_pred_svr4.reshape(y_val_pred_svr4.shape[0],1)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_svr4))
))

[RMSE] 検証用データ : 68,829.644


In [22]:
#決定木にて検証用データの予測
dtr4 = DecisionTreeRegressor()
dtr4.fit(X_train_log, y_train)
y_val_pred_dtr4 = dtr4.predict(X_validation_log)
y_val_pred_dtr4 = y_val_pred_dtr4.reshape(y_val_pred_dtr4.shape[0],1)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_dtr4))
))


[RMSE] 検証用データ : 46,613.650


In [23]:
#ブレンディング
y_val_pred_blend4 = np.mean([
    y_val_pred_lr4, 
    y_val_pred_svr4, 
    y_val_pred_dtr4
], axis=0)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_blend4))
))

[RMSE] 検証用データ : 49,410.915


**結果：精度は高まらず(決定木より精度が低い)**

### 5.SVMのカーネルの種類を混ぜ合わせる

In [24]:
### SVMにて検証用データの予測(カーネル:linear)
svr5_linear = SVR(gamma=1, kernel='linear', C=1e5, epsilon=50)
svr5_linear.fit(X_train_st, y_train.reshape(-1))
y_val_pred_svr5_linear = svr5_linear.predict(X_validation_st)
y_val_pred_svr5_linear = y_val_pred_svr5_linear.reshape(y_val_pred_svr5_linear.shape[0],1)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_svr5_linear))
))

[RMSE] 検証用データ : 51,827.790


In [25]:
#SVMにて検証用データの予測(カーネル:rbf)
svr5_rbf = SVR(gamma=1, kernel='rbf', C=1e3)
svr5_rbf.fit(X_train_st, y_train.reshape(-1))
y_val_pred_svr5_rbf = svr5_rbf.predict(X_validation_st)
y_val_pred_svr5_rbf = y_val_pred_svr5_rbf.reshape(y_val_pred_svr5_rbf.shape[0],1)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_svr5_rbf))
))

[RMSE] 検証用データ : 67,184.879


In [26]:
#SVMにて検証用データの予測(カーネル:sigmoid)
svr5_sig = SVR(gamma=0.05, kernel='sigmoid', C=1e5)
svr5_sig.fit(X_train_st, y_train.reshape(-1))
y_val_pred_svr5_sig = svr5_sig.predict(X_validation_st)
y_val_pred_svr5_sig = y_val_pred_svr5_sig.reshape(y_val_pred_svr5_sig.shape[0],1)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_svr5_sig))
))

[RMSE] 検証用データ : 53,433.656


In [27]:
#ブレンディング
y_val_pred_blend5 = np.mean([
    y_val_pred_svr5_linear, 
    y_val_pred_svr5_rbf, 
    y_val_pred_svr5_sig
], axis=0)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_blend5))
))

[RMSE] 検証用データ : 56,259.156


**結果：精度は高まらず(sigmoidより精度が低い)**

 ### 6.色々チューニングして一番精度を高める

In [28]:
#線形回帰にて検証用データの予測
lr6 = LinearRegression(fit_intercept = True, normalize = True)
lr6.fit(X_train, y_train)
y_val_pred_lr6 = lr6.predict(X_validation)

# 検証用データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_lr6))
))

[RMSE] 検証用データ : 49,955.529


In [29]:
#SVMにて検証用データの予測
svr6 = SVR(gamma=0.01, kernel='rbf', C=1e6)
svr6.fit(X_train_st, y_train.reshape(-1))
y_val_pred_svr6 = svr6.predict(X_validation_st)
y_val_pred_svr6 = y_val_pred_svr6.reshape(y_val_pred_svr6.shape[0],1)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_svr6))
))

[RMSE] 検証用データ : 47,945.397


In [30]:
#決定木にて検証用データの予測
dtr6 = DecisionTreeRegressor(criterion="mse", random_state=5, max_depth=5)
dtr6.fit(X_train, y_train)
y_val_pred_dtr6 = dtr6.predict(X_validation)
y_val_pred_dtr6 = y_val_pred_dtr6.reshape(y_val_pred_dtr6.shape[0],1)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_dtr6))
))


[RMSE] 検証用データ : 42,945.369


In [31]:
#ブレンディング
y_val_pred_blend6 = np.average([
    y_val_pred_lr6, 
    y_val_pred_svr6, 
    y_val_pred_dtr6
], axis=0, weights=[1,1,8])

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_blend6))
))

[RMSE] 検証用データ : 42,440.326


**結果：精度が高まった**

## 【問題2】バギング
バギングを実装し、単一モデルより精度があがる例を最低1つ示してください。

バギングは入力データの選び方を多様化する方法です。学習データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ブートストラップサンプル）を作り出します。それらによってモデルをN個学習し、推定結果の平均をとります。ブレンディングと異なり、それぞれの重み付けを変えることはありません。

[sklearn.model_selection.train_test_split — scikit-learn 0.20.0 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

scikit-learnのtrain_test_splitを、shuffleパラメータをTrueにして使うことで、ランダムにデータを分割することができます。これによりブートストラップサンプルが手に入ります。

推定結果の平均をとる部分はブースティングと同様の実装になります。



### ブートストラップサンプルを作成する

In [32]:
X_train01, _, y_train01, _ = train_test_split(
    X_train, y_train, test_size=0.2,  shuffle=True)

In [33]:
X_train02, _, y_train02, _ = train_test_split(
    X_train, y_train, test_size=0.2,  shuffle=True)

In [34]:
X_train03, _, y_train03, _ = train_test_split(
    X_train, y_train, test_size=0.2,  shuffle=True)

In [35]:
X_train04, _, y_train04, _ = train_test_split(
    X_train, y_train, test_size=0.2, shuffle=True)

### 1.線形回帰モデルにてバギング

In [36]:
#線形回帰にて検証用データの予測(比較対象)
lr_or = LinearRegression(fit_intercept = True, normalize = True)
lr_or.fit(X_train, y_train)
y_val_pred_lr_or = lr_or.predict(X_validation)

# 検証用データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_lr_or))
))

[RMSE] 検証用データ : 49,955.529


In [37]:
#線形回帰にて検証用データの予測
lr_bg01 = LinearRegression(fit_intercept = True, normalize = True)
lr_bg01.fit(X_train01, y_train01)
y_val_pred_lr_bg01 = lr_bg01.predict(X_validation)

# 検証用データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_lr_bg01))
))

[RMSE] 検証用データ : 50,347.794


In [38]:
#線形回帰にて検証用データの予測
lr_bg02 = LinearRegression(fit_intercept = True, normalize = True)
lr_bg02.fit(X_train02, y_train02)
y_val_pred_lr_bg02 = lr_bg02.predict(X_validation)

# 検証用データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_lr_bg02))
))

[RMSE] 検証用データ : 49,583.841


In [39]:
#線形回帰にて検証用データの予測
lr_bg03 = LinearRegression(fit_intercept = True, normalize = True)
lr_bg03.fit(X_train03, y_train03)
y_val_pred_lr_bg03 = lr_bg03.predict(X_validation)

# 検証用データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_lr_bg03))
))

[RMSE] 検証用データ : 49,979.229


In [40]:
#線形回帰にて検証用データの予測
lr_bg04 = LinearRegression(fit_intercept = True, normalize = True)
lr_bg04.fit(X_train04, y_train04)
y_val_pred_lr_bg04 = lr_bg04.predict(X_validation)

# 検証用データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_lr_bg04))
))

[RMSE] 検証用データ : 49,464.220


In [41]:
#バギング
y_val_pred_lr_bg = np.average([
    y_val_pred_lr_bg01, 
    y_val_pred_lr_bg02, 
    y_val_pred_lr_bg03, 
    y_val_pred_lr_bg04
], axis=0)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_lr_bg))
))

[RMSE] 検証用データ : 49,788.100


**結果：精度上がらず**

### 2.SVMにてバギング

In [42]:
#SVMにて検証用データの予測（比較用）
svr_or = SVR(gamma=0.01, kernel='linear', C=1e3)
svr_or.fit(X_train, y_train.reshape(-1))
y_val_pred_svr_or = svr_or.predict(X_validation)
y_val_pred_svr_or = y_val_pred_svr_or.reshape(y_val_pred_svr_or.shape[0],1)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_svr_or))
))

[RMSE] 検証用データ : 51,794.306


In [43]:
#SVMにて検証用データの予測
svr_bg01 = SVR(gamma=0.01, kernel='linear', C=1e3)
svr_bg01.fit(X_train01, y_train01.reshape(-1))
y_val_pred_svr_bg01 = svr_bg01.predict(X_validation)
y_val_pred_svr_bg01 = y_val_pred_svr_bg01.reshape(y_val_pred_svr_bg01.shape[0],1)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_svr_bg01))
))

[RMSE] 検証用データ : 52,036.682


In [44]:
#SVMにて検証用データの予測
svr_bg02 = SVR(gamma=0.01, kernel='linear', C=1e3)
svr_bg02.fit(X_train02, y_train02.reshape(-1))
y_val_pred_svr_bg02 = svr_bg02.predict(X_validation)
y_val_pred_svr_bg02 = y_val_pred_svr_bg02.reshape(y_val_pred_svr_bg02.shape[0],1)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_svr_bg02))
))

[RMSE] 検証用データ : 51,689.700


In [45]:
#SVMにて検証用データの予測
svr_bg03 = SVR(gamma=0.01, kernel='linear', C=1e3)
svr_bg03.fit(X_train03, y_train03.reshape(-1))
y_val_pred_svr_bg03 = svr_bg03.predict(X_validation)
y_val_pred_svr_bg03 = y_val_pred_svr_bg03.reshape(y_val_pred_svr_bg03.shape[0],1)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_svr_bg03))
))

[RMSE] 検証用データ : 51,872.374


In [46]:
#SVMにて検証用データの予測
svr_bg04 = SVR(gamma=0.01, kernel='linear', C=1e3)
svr_bg04.fit(X_train04, y_train04.reshape(-1))
y_val_pred_svr_bg04 = svr_bg04.predict(X_validation)
y_val_pred_svr_bg04 = y_val_pred_svr_bg04.reshape(y_val_pred_svr_bg04.shape[0],1)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_svr_bg04))
))

[RMSE] 検証用データ : 52,121.186


In [47]:
#バギング
y_val_pred_svr_bg = np.average([
    y_val_pred_svr_bg01, 
    y_val_pred_svr_bg02, 
    y_val_pred_svr_bg03, 
    y_val_pred_svr_bg04
], axis=0)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_svr_bg))
))

[RMSE] 検証用データ : 51,922.062


**結果：元データと比較すると精度は向上した**

### 3.決定木でバギング

In [48]:
#決定木にて検証用データの予測（比較検証用）
dtr_or = DecisionTreeRegressor(criterion="mse", random_state=5, max_depth=5)
dtr_or.fit(X_train, y_train)
y_val_pred_dtr_or = dtr_or.predict(X_validation)
y_val_pred_dtr_or = y_val_pred_dtr_or.reshape(y_val_pred_dtr_or.shape[0],1)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_dtr_or))
))


[RMSE] 検証用データ : 42,945.369


In [49]:
#決定木にて検証用データの予測（比較検証用）
dtr_bg01 = DecisionTreeRegressor(criterion="mse", random_state=5, max_depth=5)
dtr_bg01.fit(X_train01, y_train01)
y_val_pred_dtr_bg01 = dtr_bg01.predict(X_validation)
y_val_pred_dtr_bg01 = y_val_pred_dtr_bg01.reshape(y_val_pred_dtr_bg01.shape[0],1)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_dtr_bg01))
))


[RMSE] 検証用データ : 56,000.226


In [50]:
#決定木にて検証用データの予測（比較検証用）
dtr_bg02 = DecisionTreeRegressor(criterion="mse", random_state=5, max_depth=5)
dtr_bg02.fit(X_train02, y_train02)
y_val_pred_dtr_bg02 = dtr_bg02.predict(X_validation)
y_val_pred_dtr_bg02 = y_val_pred_dtr_bg02.reshape(y_val_pred_dtr_bg02.shape[0],1)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_dtr_bg02))
))


[RMSE] 検証用データ : 44,152.706


In [51]:
#決定木にて検証用データの予測（比較検証用）
dtr_bg03 = DecisionTreeRegressor(criterion="mse", random_state=5, max_depth=5)
dtr_bg03.fit(X_train03, y_train03)
y_val_pred_dtr_bg03 = dtr_bg03.predict(X_validation)
y_val_pred_dtr_bg03 = y_val_pred_dtr_bg03.reshape(y_val_pred_dtr_bg03.shape[0],1)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_dtr_bg03))
))


[RMSE] 検証用データ : 45,935.982


In [52]:
#決定木にて検証用データの予測（比較検証用）
dtr_bg04 = DecisionTreeRegressor(criterion="mse", random_state=5, max_depth=5)
dtr_bg04.fit(X_train04, y_train04)
y_val_pred_dtr_bg04 = dtr_bg04.predict(X_validation)
y_val_pred_dtr_bg04 = y_val_pred_dtr_bg04.reshape(y_val_pred_dtr_bg04.shape[0],1)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_dtr_bg04))
))


[RMSE] 検証用データ : 42,102.528


In [53]:
#バギング
y_val_pred_dtr_bg = np.average([
    y_val_pred_dtr_bg01, 
    y_val_pred_dtr_bg02, 
    y_val_pred_dtr_bg03, 
    y_val_pred_dtr_bg04
], axis=0)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.3f}'.format(
    np.sqrt(mean_squared_error(y_validation, y_val_pred_dtr_bg))
))

[RMSE] 検証用データ : 43,075.844


**結果：精度向上した**

## 【問題3】スタッキング
**ここから説明変数・目的変数両者に対して対数変換を行う**  

スタッキングを実装し、単一モデルより精度があがる例を最低1つ示してください。

スタッキングの手順は以下の通りです。最低限ステージ0とステージ1があればスタッキングは成立するため、それを実装してください。まずは$K_0=3,M_0=2$程度にします。

**学習時**

（ステージ $0$ ）

- 学習データを$K_0$個に分割する。
- 分割した内の $(K_0 - 1)$ 個をまとめて学習用データ、残り $1$ 個を推定用データとする組み合わせが $K_0$ 個作れる。
- あるモデルのインスタンスを $K_0$ 個用意し、異なる学習用データを使い学習する。
- それぞれの学習済みモデルに対して、使っていない残り $1$ 個の推定用データを入力し、推定値を得る。（これをブレンドデータと呼ぶ）
- さらに、異なるモデルのインスタンスも $K_0$ 個用意し、同様のことを行う。モデルが $M_0$個あれば、 $M_0$ 個のブレンドデータが得られる。


（ステージ $n$ ）

- ステージ $n-1$ のブレンドデータを$M_{n-1}$次元の特徴量を持つ学習用データと考え、 $K_n$個に分割する。以下同様である。


（ステージ $N$ ）＊最後のステージ

- ステージ $N-1$ の $M_{N-1}$ 個のブレンドデータを$M_{N-1}$ 次元の特徴量の入力として、1種類のモデルの学習を行う。これが最終的な推定を行うモデルとなる。


**推定時**

（ステージ $0$ ）

- テストデータを $K_0×M_0$ 個の学習済みモデルに入力し、$K_0×M_0$個の推定値を得る。これを $K_0$ の軸で平均値を求め $M_0$ 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）


（ステージ $n$ ）

- ステージ $n-1$ で得たブレンドテストを $K_n×M_n$ 個の学習済みモデルに入力し、$K_n×M_n$ 個の推定値を得る。これを $K_n$ の軸で平均値を求め $M_0$ 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）


（ステージ $N$ ）＊最後のステージ

- ステージ$N-1$ で得たブレンドテストを学習済みモデルに入力し、推定値を得る。


### 動きを確認する

#### 学習ステージ0

In [54]:
#インデックスを三分割
X_idx = np.random.permutation(np.arange(0, X_train_log.shape[0], 1))
f0_idx, f1_idx, f2_idx = np.array_split(X_idx, 3)
f1_f2_idx = np.concatenate([f1_idx, f2_idx])
f0_f2_idx = np.concatenate([f0_idx, f2_idx])
f0_f1_idx = np.concatenate([f0_idx, f1_idx])

In [55]:
#f0_idxに対するブレンドデータ0を作成
s0_reg00 = DecisionTreeRegressor(criterion="mse", random_state=5, max_depth=3)
s0_reg00.fit(X_train_log[f1_f2_idx], y_train_log[f1_f2_idx])
brend_data_reg00 = s0_reg00.predict(X_train_log[f0_idx])
brend_data_reg00 = brend_data_reg00.reshape(brend_data_reg00.shape[0],1)

#f1_idxに対するブレンドデータ0を作成
s0_reg01 = DecisionTreeRegressor(criterion="mse", random_state=5, max_depth=3)
s0_reg01.fit(X_train_log[f0_f2_idx], y_train_log[f0_f2_idx])
brend_data_reg01 = s0_reg01.predict(X_train_log[f1_idx])
brend_data_reg01 = brend_data_reg01.reshape(brend_data_reg01.shape[0],1)

#f2_idxに対するブレンドデータを作成
s0_reg02 = DecisionTreeRegressor(criterion="mse", random_state=5, max_depth=3)
s0_reg02.fit(X_train_log[f0_f1_idx], y_train_log[f0_f1_idx])
brend_data_reg02 = s0_reg02.predict(X_train_log[f2_idx])
brend_data_reg02 = brend_data_reg02.reshape(brend_data_reg02.shape[0],1)


In [56]:
#f0_idxに対するブレンドデータを作成
s0_reg10 = LinearRegression()
s0_reg10.fit(X_train_log[f1_f2_idx], y_train_log[f1_f2_idx])
brend_data_reg10 = s0_reg10.predict(X_train_log[f0_idx])
brend_data_reg10 = brend_data_reg10.reshape(brend_data_reg10.shape[0],1)


#f1_idxに対するブレンドデータを作成
s0_reg11 = LinearRegression()
s0_reg11.fit(X_train_log[f0_f2_idx], y_train_log[f0_f2_idx])
brend_data_reg11 = s0_reg11.predict(X_train_log[f1_idx])
brend_data_reg11 = brend_data_reg11.reshape(brend_data_reg11.shape[0],1)


#f2_idxに対するブレンドデータを作成
s0_reg12 = LinearRegression()
s0_reg12.fit(X_train_log[f0_f1_idx], y_train_log[f0_f1_idx])
brend_data_reg12 = s0_reg12.predict(X_train_log[f2_idx])
brend_data_reg12 = brend_data_reg12.reshape(brend_data_reg12.shape[0],1)


In [57]:
 #K0個のbrend_data_reg0に格納
brend_data_reg0 = np.zeros( X_train_log.shape[0]).reshape(-1, 1)
brend_data_reg0[f0_idx] = brend_data_reg00
brend_data_reg0[f1_idx] = brend_data_reg01
brend_data_reg0[f2_idx] = brend_data_reg02

#K0個brend_data_reg1を結合
brend_data_reg1 = np.zeros( X_train_log.shape[0]).reshape(-1, 1)
brend_data_reg1[f0_idx] = brend_data_reg10
brend_data_reg1[f1_idx] = brend_data_reg11
brend_data_reg1[f2_idx] = brend_data_reg12

#上記を結合
brend_data_s0 = np.concatenate([brend_data_reg0, brend_data_reg1], axis=1)

#### 学習ステージ1

In [58]:
#インデックスを三分割
X_idx = np.random.permutation(np.arange(0, X_train_log.shape[0], 1))
f0_idx, f1_idx, f2_idx = np.array_split(X_idx, 3)
f1_f2_idx = np.concatenate([f1_idx, f2_idx])
f0_f2_idx = np.concatenate([f0_idx, f2_idx])
f0_f1_idx = np.concatenate([f0_idx, f1_idx])

In [59]:
 #ステージ0のデータで再度学習
#f0_idxに対するブレンドデータを作成
s1_reg00 = LinearRegression()
s1_reg00.fit(brend_data_s0[f1_f2_idx], y_train_log[f1_f2_idx])
brend_data_s1_reg00 = s1_reg00.predict(brend_data_s0[f0_idx])

#f1_idxに対するブレンドデータを作成
s1_reg01 = LinearRegression()
s1_reg01.fit(brend_data_s0[f0_f2_idx], y_train_log[f0_f2_idx])
brend_data_s1_reg01 = s1_reg01.predict(brend_data_s0[f1_idx])

#f2_idxに対するブレンドデータを作成
s1_reg02 = LinearRegression()
s1_reg02.fit(brend_data_s0[f0_f1_idx], y_train_log[f0_f1_idx])
brend_data_s1_reg02 = s1_reg02.predict(brend_data_s0[f2_idx])

In [60]:
 #K1個のbrend_dataを結合
brend_data_s1_reg0 = np.zeros( X_train_log.shape[0]).reshape(-1, 1)
brend_data_s1_reg0[f0_idx] = brend_data_s1_reg00
brend_data_s1_reg0[f1_idx] = brend_data_s1_reg01
brend_data_s1_reg0[f2_idx] = brend_data_s1_reg02

#### 学習ステージ2

In [61]:
#推定用のインスタンス・モデルの生成
s2_reg = LinearRegression()
s2_reg.fit(brend_data_s1_reg0, y_train_log)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

#### 推定ステージ0

In [62]:
#学習ステージ0にて学習したモデルs0_reg0*にてpredict
brend_test_reg00 = s0_reg00.predict(X_validation_log)
brend_test_reg01 = s0_reg01.predict(X_validation_log)
brend_test_reg02 = s0_reg02.predict(X_validation_log)

#平均を取得する
brend_test_reg0 = np.mean([
    brend_test_reg00, 
    brend_test_reg01, 
    brend_test_reg02], axis=0).reshape(-1,1)

#学習ステージ0にて学習したモデルs0_reg1*にてpredict
brend_test_reg10 = s0_reg10.predict(X_validation_log)
brend_test_reg11 = s0_reg11.predict(X_validation_log)
brend_test_reg12 = s0_reg12.predict(X_validation_log)

#平均を取得する
brend_test_reg1 = np.mean([
    brend_test_reg10, 
    brend_test_reg11, 
    brend_test_reg12], axis=0).reshape(-1,1)

#上記を結合する
brend_test_s0 = np.concatenate([brend_test_reg0, brend_test_reg1], axis=1)

#### 推定ステージ1

In [63]:
#学習ステージ1にて学習したモデルs1_reg0*にてpredict
brend_test_s1_reg00 = s1_reg00.predict(brend_test_s0)
brend_test_s1_reg01 = s1_reg01.predict(brend_test_s0)
brend_test_s1_reg02 = s1_reg02.predict(brend_test_s0)

#平均を取得する
brend_test_s1_reg0 = np.mean([
    brend_test_s1_reg00, 
    brend_test_s1_reg01, 
    brend_test_s1_reg02], axis=0).reshape(-1,1)



#### 推定ステージ2

In [64]:
#推定ステージ1にて取得した
brend_test_s2 = s2_reg.predict(brend_test_s1_reg0)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.4f}'.format(
    np.sqrt(mean_squared_error(y_validation_log, brend_test_s2))
))

[RMSE] 検証用データ : 0.2275


#### 線形回帰での平均二乗誤差

In [65]:
lr = LinearRegression()
lr.fit(X_train_log, y_train_log)
y_pred_lr = lr.predict(X_validation_log)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.4f}'.format(
    np.sqrt(mean_squared_error(y_validation_log, y_pred_lr))
))

[RMSE] 検証用データ : 0.2282


#### 決定木での平均二乗誤差

In [66]:
dtr = DecisionTreeRegressor(criterion="mse", random_state=5, max_depth=3)
dtr.fit(X_train_log, y_train_log)
y_pred_dtr = dtr.predict(X_validation_log)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.4f}'.format(
    np.sqrt(mean_squared_error(y_validation_log, y_pred_dtr))
))

[RMSE] 検証用データ : 0.2563


### 結果
スタッキングにより精度が向上した

### クラスを作成する（写経）
https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard

In [76]:
#ライブラリのインポート
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

In [68]:
#lightgbmのインスタンス生成
model_lgb = lgb.LGBMRegressor(
    objective='regression', 
    num_leabes=5, 
    learning_rate=0.05, 
    n_estimators=720, 
    max_bin=55,
    bagging_fraction=0.8,
    bagging_freq=5, 
    feature_fraction=0.2319, 
    feature_fraction_seed=9,
    bagging_seed=9, 
    min_data_in_leaf=6, 
    min_sum_hessian_in_leaf=11
                             )

In [69]:
#決定木のインスタンス生成
model_dtr = DecisionTreeRegressor(
    criterion="mse", 
    random_state=5, 
    max_depth=5
)

In [70]:
#線形回帰のインスタンス生成
model_lr = LinearRegression()

In [71]:
#モデル同士の平均にて学習・予測を行う(≒blending)
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    #データを収めるためのモデルを複製する
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        #複製したモデルにて学習を行う
        for model in self.models_:
            model.fit(X, y)
            
        return self
    
    #複製したモデルを用いて予測し、平均する
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)
        
        
        

In [72]:
#線形回帰、決定木、lgbmの平均モデルのインスタンス生成
averaged_models = AveragingModels(models = (model_lr, model_dtr, model_lgb))

#学習データにてモデル作成
averaged_models.fit(X_train_log, y_train_log.reshape(-1))

#検証用データにて予測
ave_pred = averaged_models.predict(X_validation_log)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.4f}'.format(
    np.sqrt(mean_squared_error(y_validation_log, ave_pred))
))

[RMSE] 検証用データ : 0.2167


In [73]:
#スタッキングのクラス作成（ステージ0とステージ1の構造）
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    
    def  __init__(self, base_models, meta_model, n_folds=3):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
    
    #学習
    def fit(self, X, y):
        #ベースモデル(ステージ0)のモデルより複製を行う
        self.base_models_ = [list() for x in self.base_models]
        
        #メタモデルの複製
        self.meta_model_ = clone(self.meta_model)
        
        #データ分割
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        #新たな特徴量とするout_of_fold_predictionsを作成する
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        
        #それぞれのベースモデルにて学習、推定を行い、out_of_fold_predictionsに格納
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        #新しい特徴量out-of-fold-predictionsを用いて複製したメタモデルを学習する
        self.meta_model_.fit(out_of_fold_predictions, y)
        
        return self
    
    #推定
    def predict(self, X):
        #ベースモデルより新しい特徴量を作成した上で、メタモデルで予測を行う
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_
        ])

        return self.meta_model_.predict(meta_features)
        

In [74]:
#スタッキングモデルにてインスタンス生成
stacked_averaged_models = StackingAveragedModels(
    base_models = (model_dtr, model_lgb), 
    meta_model = model_lr
)

#学習を行う
stacked_averaged_models.fit(X_train_log, y_train_log.reshape(-1))

#予測する
stacked_pred = stacked_averaged_models.predict(X_validation_log)

#検証データに関して平均二乗誤差を出力
print('[RMSE] 検証用データ : {:,.4f}'.format(
    np.sqrt(mean_squared_error(y_validation_log, stacked_pred))
))


[RMSE] 検証用データ : 0.2192


In [75]:
model_lgb.fit(X_train_log, y_train_log.reshape(-1))

lgb_pred = model_lgb.predict(X_validation_log)

print('[RMSE] 検証用データ : {:,.4f}'.format(
    np.sqrt(mean_squared_error(y_validation_log, lgb_pred))
))

[RMSE] 検証用データ : 0.2254


### 結果
LightGBMを加えたスタッキングでも精度が向上した

## 【問題4】（アドバンス課題）Microsoft Malware Prediction
時間的に余裕がある場合は、KaggleのMalwareコンペを題材に、アンサンブル学習を実際に利用してみましょう。EDAからはじめ、推定値の提出まで挑戦してください。

[Microsoft Malware Prediction | Kaggle](https://www.kaggle.com/c/microsoft-malware-prediction)
です。

注意点

非常に大きなデータセットなため、扱いに工夫が必要です。

※省略